In [ ]:
# 
# Q ) How data is ingested to your pipeline ?
# Q)  

In [ ]:
# Strucrued Streaming 
# Q) What u do for late arrival of data in your pipeline 
# Q) What is trigger point of kafka , How frequenlty u poll data from Pipeline 
# Q) What is volune of data is being processed in your pipeline 

In [ ]:
# Data Quality 
# How data is managed in your pipeline 
# What types of Checked is done on streaming vs batch processing ?

In [ ]:
# Pyspark 
# What is Volumne of data is be processed 
# What is Bucketing 
# How do you deceding bucketing , For example customers(cust_id, cust_name, ..)
# What is difference between partitioning vs bucketing 
# 

In [ ]:
# Orchestrate 
# Q) How did you orchestrate your pipeline ?
# 

In [ ]:
# SQL_Problem # or Pyspark 
'''
Input_table
-------------
product,failure_date
A , 10-AUG
A , 11-AUG
A , 12-AUG 

A , 14-AUG
A , 15-AUG 

B , 10-AUG
B , 11-AUG
B , 12-AUG
B , 13-AUG 
C , 10-AUG 
D , 11-AUG  


Output 
--------
product,min_date, max_date 
A, 10-AUG , 12-AUG 
A, 14-AUG , 15-AUG 
B, 10-AUG , 13-AUG 
C, 10-AUG , NA
D, 11-AUG , NA 
'''

## Pyspark Problem 
Got it 👍 — you’re showing a **data transformation** problem often asked in PySpark interviews.

You have **wide data** (columns `CL1`, `CL2`, `CL3`) and you need to **normalize** or **unpivot (melt)** it into a **long format** — i.e., transform columns into rows.

Let’s restate and solve step by step.

---

## 🧩 **Input Data (Wide Format)**

| ID | CL1 | CL2 | CL3 |
| -- | --- | --- | --- |
| A  | Y   |     |     |
| B  | Y   | Y   |     |
| C  | Y   | Y   | Y   |
| D  |     | Y   |     |
| E  |     |     |     |

---

## 🎯 **Expected Output (Long Format)**

| ID | CL |
| -- | -- |
| A  | 1  |
| B  | 1  |
| B  | 2  |
| C  | 1  |
| C  | 2  |
| C  | 3  |
| D  | 2  |

---

## ⚙️ **PySpark Solution**

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, when, array, explode, lit

# Create Spark session
spark = SparkSession.builder.appName("UnpivotExample").getOrCreate()

# Sample data
data = [
    ("A", "Y", None, None),
    ("B", "Y", "Y", None),
    ("C", "Y", "Y", "Y"),
    ("D", None, "Y", None),
    ("E", None, None, None)
]

columns = ["ID", "CL1", "CL2", "CL3"]

df = spark.createDataFrame(data, columns)

print("=== Original Data ===")
df.show()

# Create an array of (column index) where value == 'Y'
# Unpivot / melt logic using explode
melted_df = (
    df.withColumn(
        "CL_array",
        array(
            when(df.CL1 == "Y", lit(1)),
            when(df.CL2 == "Y", lit(2)),
            when(df.CL3 == "Y", lit(3))
        )
    )
    .withColumn("CL", explode(expr("filter(CL_array, x -> x is not null)")))
    .select("ID", "CL")
)

print("=== Transformed Data ===")
melted_df.show()
```

---

### 🧠 **Explanation**

| Step                 | What Happens                                                         |
| -------------------- | -------------------------------------------------------------------- |
| `array()`            | Combine CL1, CL2, CL3 into a single array of possible column indices |
| `when()`             | Only keep column index if value is 'Y'                               |
| `filter()`           | Remove nulls from the array                                          |
| `explode()`          | Convert each element in array to a new row                           |
| `select("ID", "CL")` | Keep final columns                                                   |

---

### ✅ **Output**

```
+---+---+
| ID| CL|
+---+---+
|  A|  1|
|  B|  1|
|  B|  2|
|  C|  1|
|  C|  2|
|  C|  3|
|  D|  2|
+---+---+
```

---

### 💡 **Interview Tip:**

This is called **unpivoting** or **normalizing** a DataFrame.

In PySpark, there’s no direct `melt()` like in pandas, so you use:

* `array()` + `explode()` for manual unpivot
  or
* SQL `UNION ALL` if columns are few (less dynamic but simpler).

---

Would you like me to show the **SQL-style approach** as well (using `UNION ALL`) — it’s simpler and interviewers often ask both versions for comparison?
